<a href="https://colab.research.google.com/github/amyth18/CS598-Deep-Learning-Final-Project/blob/main/CS598_Deep_Learning_For_Healthcare_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

In this notebook we provide our implementation for a disease prediction model based on discharge summary text. The implementation is based on the paper titled "[A disease inference method based on symptom extraction and bidirectional Long Short Term Memory networks](https://https://pubmed.ncbi.nlm.nih.gov/31301375/)" by Donglin Guo et al.

The notebook is divided in 4 parts.
1. Data Preprocessing
2. Feature Engineering
3. Model Training
4. Model Evaluation

The data preprocessing step needs to be performed only once and the output of this step can be used to run the other sections any number of times.

We have tested this notebook in Google Colab pro+ environment. We highly recommend using GPU when training the model.

We have used the MIMIC III dataset for training and testing our model, we specifically we need the `NOTEVENTS.csv` and `DIAGNOSES_ICD.csv` files to make this model work.

We have used Google Drive as our presistence layer. If you want to use any other storage then you will have to modify the storage paths in the cells accordingly.

# Initial Setup

Load the required packages

In [ ]:
! pip install gensim --upgrade
! pip install psutil
! pip install transformers

In [85]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import pickle

Mount the Google Drive into colab environment. The Google Drive contains our dataset as well acts as persistence for intermdiate data, model and results.

In [86]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Setup the base path of the project

In [87]:
PROJECT_PATH = "/content/drive/My Drive/DLH Final Project/"

# Load Data

We are primarily interested in the following 2 files/tables from the MIMIC III dataset.
1.   NOTESEVENTS.csv (context discharge summary in "TEXT" field for each hospital admission.)
2.   DIAGNOSES_ICD.csv (contains ICD9 codes for conditions diagnosed in an admimission)



In [88]:
df = pd.read_csv("/content/drive/My Drive/DLH Final Project/mimic3/NOTEEVENTS.csv")
df[["SUBJECT_ID", "HADM_ID", "TEXT"]].head(3)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,SUBJECT_ID,HADM_ID,TEXT
0,22532,167853.0,Admission Date: [**2151-7-16**] Dischar...
1,13702,107527.0,Admission Date: [**2118-6-2**] Discharg...
2,13702,167118.0,Admission Date: [**2119-5-4**] D...


In [89]:
df_icd_codes = pd.read_csv(
    "/content/drive/My Drive/DLH Final Project/mimic3/DIAGNOSES_ICD.csv")

df_icd_codes[["SUBJECT_ID", "HADM_ID", "ICD9_CODE"]].head(3)

,SUBJECT_ID,HADM_ID,ICD9_CODE
0,109,172335,40301
1,109,172335,486
2,109,172335,58281


# Data Preprocessing

In this section we perform the following pre-processing steps on out data set and produce a dataframe that is then used for feature engineering in the next section.

1.   We first find the top #50 ICD codes (i.e top 50 conditions that are commonly diagnosed in admissions)

2.   We then select only those adminisions from `df_icd_codes` dataframe that contain atleat one of the top #50 ICD9 codes.

3. We then perform inner join on the filtered `df_icd_codes` to the `df` (i.e. NOTEVENTS.csv) to select discharge summary text from only those admimissions that contain at least one top #50 ICD9 code.

4. We then generate Word2Vec embeddings using all the filtered discharge summaries using Gensim. This will be used later in feature engineering.

5. We then extract only the symptom phrases from the discharge summaries using a pre-trained BERT model. The symptoms will then form the input features (represented in 2 forms) for our prediction model. This step is performed Feature Engineering section.

The following diagram shows the data pre-processing pipeline.

![picture](https://drive.google.com/file/d/1deg7uoKN9cd7LWleztiWTLCgBKg2r1KT/view?usp=sharing)

## Utility Routines For Data Pre-Processing

Performs data cleaning, curation and symptom extraction using pre-trained BERT model on clinical text.

In [90]:
from typing import List
import re
import nltk
from transformers import AutoTokenizer, pipeline,  AutoModelForTokenClassification

from nltk.corpus import stopwords
nltk.download('stopwords')

eng_stop_words =  stopwords.words('english')

class MySentences(object):
    def __init__(self, dframe):
        self.dframe = dframe
        tokenizer = AutoTokenizer.from_pretrained(
            "samrawal/bert-base-uncased_clinical-ner")
        model = AutoModelForTokenClassification.from_pretrained(
            "samrawal/bert-base-uncased_clinical-ner")
        symptom_extractor = pipeline('ner', model=model, tokenizer=tokenizer,
                                     device=0)
        text_anno = symptom_extractor(self.dframe["TEXT"].tolist(), 
                                           batch_size=256)
        self.data = list(zip(text_anno, self.dframe["TEXT"].tolist()))        

    # TODO: Keeping only alpha numeric characters and spaces for now.
    # need to make this better. Find some good libraries.
    def sanitize_text(self, text):
      test = text.strip()
      text = re.sub(r'\s\s+', ' ', text)
      text = re.sub(r'[^a-zA-z0-9\/\.\?\!\s;,\'\-]', '', text)
      text = re.sub(r'[\.\-\/\?\!;,]', ' ', text)
      text = re.sub(r'[\[\]]', '', text)
      return text

    def extract_symptoms(self, input_anno_text) -> List[str]:
      """
      The method extract the symptom phrases from the input
      """
      extractions, text = input_anno_text
      span_extract = []
      for extract in extractions:
        if 'problem' in extract['entity']:
          span_extract.append((extract['start'], extract['end']))

      # Check if this span_extract is empty
      if not span_extract:
        return []
      
      span_st = span_extract[0][0]
      final_span = []
      final_end = span_extract[0][1]
      for idx, (st, end) in enumerate(span_extract):
        if idx == 0:
          continue
        if st - span_extract[idx-1][1] <= 2:
          final_end = end
          if idx == len(span_extract) - 1:
            final_span.append((span_st, final_end))
        else:
          final_span.append((span_st, final_end))
          span_st = st
          final_end = end

      text_extracts = [text[st:end].replace("\n", " ") for st, end in final_span]
      return text_extracts

    # TODO: adding some basic checks again need to make it better.
    def sanitize_words(self, sentence):
      return [w for w in sentence if w not in eng_stop_words and not w.isdigit()]

    def __iter__(self):
        for idx in range(len(self.dframe)):
          symptoms = " ".join(self.extract_symptoms(self.data[idx]))
          text = self.sanitize_text(symptoms)
          yield self.sanitize_words(text.split())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Generate Word2Vec Embeddings

We generate Word2Vec embeddings on the raw discharge summary text. These embeddings will be used later when we extract only the symptoms from the dischage summaries and generate Word2Vec embedding for those symptoms. 

Note: This step needs to be done only once before start the of project.

In [91]:
# NOTE: commenting this part so that we dont run this by mistake.
# we only need to perform this once at the start of the project.

# import gensim
# sgen = MySentences(df_dataset) # a memory-friendly iterator
# model = gensim.models.Word2Vec(sgen, min_count=5, workers=4, sample=1e-05)
# model.save("/content/drive/My Drive/DLH Final Project/mimic3/word2vec-4.model")

## Data Filtering and Tranformation

Get top #50 ICD9 codes 

In [92]:
counts = df_icd_codes["ICD9_CODE"].value_counts().head(50)
top_icd_codes = counts.index.to_list()

In [93]:
sorted_top_icd_codes = sorted(top_icd_codes)
icd_code_to_idx = dict((k, v) for v, k in enumerate(sorted_top_icd_codes))

Filter data to include admimission with top 50 diseases only and group and reorganize data in the following format <subject_id, hadm_id, [icd_code1, icd_code2 ...]>

In [94]:
df_admissions_with_top_diseases = \
df_icd_codes[df_icd_codes["ICD9_CODE"].isin(top_icd_codes)]

df_admissions_with_top_diseases = \
df_admissions_with_top_diseases.groupby(
['SUBJECT_ID', 'HADM_ID'])['ICD9_CODE'].apply(
        list).to_frame().reset_index()

Now select discharge summaries for the admimissions that contain atleast one of the top #50 ICD codes.

In [95]:
df_dataset = pd.merge(df, df_admissions_with_top_diseases, 
                       on=["SUBJECT_ID", "HADM_ID"])

df_dataset = df_dataset[df_dataset["CATEGORY"] 
                          == 'Discharge summary'].reset_index()
# free up some memory
# del df

So our dataset now looks like this.

In [96]:
df_dataset[["SUBJECT_ID", "HADM_ID", "TEXT", "ICD9_CODE"]].head(3)

,SUBJECT_ID,HADM_ID,TEXT,ICD9_CODE
0,22532,167853.0,Admission Date: [**2151-7-16**] Dischar...,"[42731, 2762, 5070, 5119]"
1,22532,167853.0,Admission Date: [**2151-7-16**] Dischar...,"[42731, 2762, 5070, 5119]"
2,13702,107527.0,Admission Date: [**2118-6-2**] Discharg...,"[51881, 486, 2761, 2449, 311]"


Now, extract only the phrases related to symptoms using pre-trained BERT model on clinical test from the discharge summary. In the paper the authoes have used MetaMap for this, we could not access MetaMap in time so we are using an alternate method. We intend to run our experiment with MetaMap during our final implementation.



In [97]:
from tqdm import tqdm
sgen = MySentences(df_dataset)
symptom_col = list()
for s in tqdm(sgen):
  symptom_col.append(s)

# add the new column to the dataset.
df_dataset["SYMPTOMS"] = symptom_col

55988it [00:10, 5203.65it/s]


In [143]:
df_dataset = df_dataset[df_dataset.apply(lambda x: len(x.SYMPTOMS) > 0, axis=1)]
df_dataset = df_dataset.reset_index()
df_dataset.to_csv("/content/drive/My Drive/DLH Final Project/kbr_df_dataset.csv")

del sgen

So our final dataset after all the pre-processing looks like this.

In [118]:
df_dataset[["SUBJECT_ID", "HADM_ID", "SYMPTOMS", "ICD9_CODE"]].head(3)

,SUBJECT_ID,HADM_ID,SYMPTOMS,ICD9_CODE
0,22532,167853.0,"[cavitary, lesions, left, lung, apex, infectio...","[42731, 2762, 5070, 5119]"
1,22532,167853.0,"[productive, cough, lb, weight, loss, shortnes...","[42731, 2762, 5070, 5119]"
2,13702,107527.0,"[emphysema, shortness, breath, COPD, flare, Fe...","[51881, 486, 2761, 2449, 311]"


# Feature Engineering

The data preprocessing steps need to be performed only once during the project (unless of course we change any data preprocessing steps). Now we load the pre-processed data and start constructing our 2 forms of feature vectors for each symptom extracted as mentioned in the paper.

## Construct Dataset With Word2Vec embeddings

In this section we generate our first form of vector representation for symptoms. We generate a `Word2Vec` embedding (using the Gensim model we trained previously in the data pre-processing step) for each symptom extracted from the discharge summary. Each input then becomes a sequence of `Word2Vec` embedding.

`[emb1, emb2 ...]`

In [119]:
from gensim.models import Word2Vec
model = Word2Vec.load('/content/drive/My Drive/DLH Final Project/mimic3/word2vec-4.model')

In [120]:
X_word2vec = list()
for idx in range(len(df_dataset)):
  # ignore words in not vocabulary
  symptoms = df_dataset["SYMPTOMS"][idx]
  symptoms_emb = [model.wv[s] for s in symptoms if s in model.wv]
  X_word2vec.append(symptoms_emb)


## Construct Dataset With TF-IDF encoding

Here we generate our second form for symptom representation. We compute a TF-IDF like metric for each symptom extracted and then represent each symtom with its TF-IDF vector. So our input in this form looks like a sequence of TF-IDF vectors.

`[td-idf(s1), tf-idf(s2) ...]`

where each `tf-idf` vector is of length of 50 (for top#50 diseases) and `s1, s2 ...` are symtoms.

In [121]:
import numpy as np
import itertools

vocab_size = len(model.wv)
tf = np.zeros((len(model.wv), len(top_icd_codes)))


for idx in range(len(df_dataset)):
  # XXX: TODO currently we treat all tokens from "TEXT" as sypmtoms
  # get the icd codes for this discharge summary
  symptoms = df_dataset['SYMPTOMS'][idx]
  icd_codes = df_dataset['ICD9_CODE'][idx]
  # create a cross product of symptoms and icd codes
  # and update tf matrix. tf matrix keeps count of how many 
  # (i.e frequency) times <symptom, icd code> pair occur in our dataset.
  for pair in itertools.product(symptoms, icd_codes):
    # update count of each (symptom, icd_code) pair to compute TF
    if pair[0] in model.wv:
      tf[model.wv.get_index(pair[0])][icd_code_to_idx[pair[1]]] += 1

# Complete the TF-IDF matrix computation.
# Compute the number of ICD Codes (i.e diseaes) each 
# symptom is associated with.
D_i = np.sum(tf > 0, axis=1)
print(D_i.shape)

log_N_Di = np.log(len(top_icd_codes)/D_i)
tf_idf = (tf.T * log_N_Di).T
print(tf_idf.shape)

(64259,)
(64259, 50)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in multiply


Now we use the above generated `tf-idf` matrix to convert each symtom into a `TF-IDF` vector.

In [122]:
# build the X_tfidf dataset
X_tf_idf = list()
for idx in range(len(df_dataset)):
  symptoms = df_dataset["SYMPTOMS"][idx]
  # get tf-idf vector for each symptom
  # ignore words in not vocabulary
  symptoms_tf_idf = [tf_idf[model.wv.get_index(s)] \
                     for s in symptoms if s in model.wv]
  X_tf_idf.append(symptoms_tf_idf)


## Construct Y (as Multi-hot Encoding)

As this is a multi-label classification problem (i.e predicting one or more diseases from top #50 most commonly occuring diseases) we use a multi-hot representation for our response variable.

We covert the ICD9 column to multi-hot encoding, we keep the old column with list of codes and add a new column with multi-hot encoding representation.

In [123]:
# new col to be added to dataframe
multi_hot_ecoding_col = list()
for idx in range(len(df_dataset)):
  icd_codes = df_dataset.iloc[idx]['ICD9_CODE']
  encoding = [0] * 50
  for code in icd_codes:
    encoding[icd_code_to_idx[code]] = 1    
  multi_hot_ecoding_col.append(encoding)

# new add a new column with multi-hot encoding.
df_dataset['ICD9_CODE_ENCODED'] = multi_hot_ecoding_col

In [124]:
# multi-hot encoding for ICD codes diagnosed.
y = df_dataset['ICD9_CODE_ENCODED'].to_list()

In [125]:
print(len(X_word2vec))
print(len((X_tf_idf)))
print(len(y))

54939
54939
54939


## Define Dataset and DataLoaders

Utility to created padded dataset so that we an perform batch processing of data in model traning and evaluation.

In [126]:
def pad_dataset(dataset, vec_size):
  seq_lengths = list()

  for idx in range(len(dataset)):
    seq_lengths.append(len(dataset[idx]))
  max_seq_length = max(seq_lengths)

  padded_dataset = torch.zeros([len(dataset), max_seq_length, vec_size], 
                               dtype=torch.float)
  for i in range(len(dataset)):
    for j in range(len(dataset[i])):
      padded_dataset[i][j] = torch.FloatTensor(dataset[i][j])
  
  return padded_dataset

Now we define our dataset and data loaders for both train and test dataset.

In [127]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader

def collate_fn(data):
  x_w2v, x_tidf, y_batch = zip(*data)
  x_w2v = pad_dataset(x_w2v, 100)
  x_tidf = pad_dataset(x_tidf, 50)
  y_batch = torch.FloatTensor(y_batch)
  x_w2v = x_w2v.cuda() if torch.cuda.is_available() else x_w2v
  x_tidf = x_tidf.cuda() if torch.cuda.is_available() else x_tidf
  y_batch = y_batch.cuda() if torch.cuda.is_available() else y_batch

  return x_w2v, x_tidf, y_batch

class CustomDataset(Dataset):

  def __init__(self, X_w2v, X_tfidf, y):              
    self.X_w2v = X_w2v
    self.X_tfidf = X_tfidf
    self.y = y
    
  def __len__(self):                
    return len(self.y)
    
  def __getitem__(self, index):          
    # your code here
    return self.X_w2v[index], self.X_tfidf[index], self.y[index]

dataset = CustomDataset(X_word2vec, X_tf_idf, y)

split = int(len(dataset)*0.8)
lengths = [split, len(dataset) - split]
train_dataset, test_dataset = random_split(dataset, lengths)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=400, 
                          collate_fn=collate_fn)

test_loader = DataLoader(test_dataset, shuffle=True, batch_size=400, 
                         collate_fn=collate_fn)

# Model Definition

Our model consiste of the following components


1. A BiLSTM model trained on Word2Vec representation of symptoms
2. Another BiLSTM model trained on TF-IDF representation of symptoms.
3. A weighted averate of predictions from the above models.

The following diagram shows the architecture of the model.

![picture](https://drive.google.com/file/d/1uENAQjwBme9TUeIcZDrDerLHi84W3QLA/view)



TODOs: 
1.   Add a dropout layer.





In [128]:
import torch.nn.functional as F

class BiLSTM(nn.Module):
  def __init__(self, input_dim, embedding_dim, output_dim):   
    super(BiLSTM, self).__init__()
    self.lstm = nn.LSTM(input_size=input_dim, 
                        hidden_size=embedding_dim,
                        num_layers=1,
                        bidirectional=True,
                        batch_first=True)
    
    self.linear = nn.Linear(embedding_dim*2, 
                            output_dim)
  
  def forward(self, X):
    out, (hn, cn) = self.lstm(X)    
    emb = torch.mean(out, dim=1)
    output = torch.sigmoid(self.linear(emb))
    return output

In [129]:
class DiseasePredictionModel(nn.Module):
  def __init__(self, weight=0.4):    
    super(DiseasePredictionModel, self).__init__()
    self.weight = 0.4    
    self.w2v_lstm = BiLSTM(input_dim=100, embedding_dim=50, output_dim=50)
    self.tf_idf_lstm = BiLSTM(input_dim=50, embedding_dim=50, output_dim=50)
  
  def forward(self, X_w2v, X_tidf):
    pred1 = self.w2v_lstm(X_w2v)
    pred2 = self.tf_idf_lstm(X_tidf)
    # compute the weighted average of predictions
    # from the 2 models.
    return self.weight * pred1 + (1-self.weight) * pred2

# Model Training

Utility to generate file names to store various results during training.

In [130]:
from datetime import datetime
import pytz

def get_model_file_name():
  return "/content/drive/My Drive/DLH Final Project/models/model-" + \
                  datetime.now(pytz.timezone('Asia/Kolkata')).strftime(
                      "%d-%m-%Y-%H-%M-%S")

def get_stats_file_name():
  return "/content/drive/My Drive/DLH Final Project/stats/stats-" + \
                  datetime.now(pytz.timezone('Asia/Kolkata')).strftime(
                      "%d-%m-%Y-%H-%M-%S")

def get_results_file_name():
  return "/content/drive/My Drive/DLH Final Project/stats/results-" + \
                  datetime.now(pytz.timezone('Asia/Kolkata')).strftime(
                      "%d-%m-%Y-%H-%M-%S")

Setup the model for training. We use Binary Cross Entropy as loss function and Adam optimizer with learning rate of 0.001 (as recommeded in the paper)

In [131]:
model = DiseasePredictionModel()
if torch.cuda.is_available():
  model.cuda()

loss = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Number of trainable parameters

In [132]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

111700

In [ ]:
import psutil
import time
import pickle

main_memory_usage = list()
gpu_memory_usage = list()
gpu_time = list()
train_loss = list()

for e in range(100):
  model.train()
  epoc_train_loss = 0
  main_memory_before = psutil.virtual_memory().used
  gpu_memory_before = torch.cuda.memory_allocated()
  start_time = time.time()

  # iterate over data in mini batches.
  for x_w2v, x_tidf, y_batch in train_loader:    
    model.zero_grad()
    pred = model(x_w2v, x_tidf)
    l = loss(pred, y_batch)
    l.backward()
    optimizer.step()    
    epoc_train_loss += l.item()
    
  # print epoc level training loss.
  print(f"epoc: {e}: Train Loss: {epoc_train_loss/len(train_loader)}")
  
  # collect cpu and memory stats.
  memory_used = psutil.virtual_memory().used
  gpu_memory_used = torch.cuda.memory_allocated()
  run_time = time.time() - start_time
  print(f"time: {run_time} memory_used: {memory_used} gpu_memory_used: {gpu_memory_used}")
  print("\n")

  train_loss.append(epoc_train_loss/len(train_loader))
  main_memory_usage.append(memory_used)
  gpu_memory_usage.append(gpu_memory_used)
  gpu_time.append(run_time)
  # end of one epoc

# save the model
torch.save(model.state_dict(), get_model_file_name())
# print and collect stats.
print(psutil.virtual_memory())

stats = {
    "gpu_mem": gpu_memory_usage,
    "main_mem": main_memory_usage,
    "gpu_time": gpu_time,
    "vmm_info": psutil.virtual_memory()
}

with open(get_stats_file_name(), "ab") as sfile:
  pickle.dump(stats, sfile)

# Model Evaluation

Since this is multi-class classication problem we use micro and macro averaring strategy for evaluation performance. We use the `scikit-learn ` utility to compute these metrics.

We set the prediction threshold (i.e. value above we mark a disease as present) as 0.2 as per the recommendation in the paper.

In [134]:
# model = DiseasePredictionModel()
# model.load_state_dict(
#    torch.load("/content/drive/My Drive/DLH Final Project/model"))

from sklearn.metrics import precision_recall_fscore_support

model.eval()
y_pred_all = list()
y_true_all = list()

for x_w2v, x_tidf, y_batch in test_loader:
  y_pred = model(x_w2v, x_tidf)
  y_pred = y_pred > 0.2
  y_pred_all.extend(y_pred.detach().to('cpu').numpy())
  y_true_all.extend(y_batch.detach().to('cpu').numpy())

p1, r1, f1, s1 = precision_recall_fscore_support(y_true_all, y_pred_all, 
                                                 average="micro")
print(f"Micro Averaging. Precision: {p1}, Recall: {r1}, F1 Score: {f1}")

p2, r2, f2, s2 = precision_recall_fscore_support(y_true_all, y_pred_all, 
                                                 average="macro")

print(f"Macro Averaging. Precision: {p2}, Recall: {r2}, F1 Score: {f2}")

results = {
    "micro": [p1, r1, f1],
    "macro": [p2, r2, f2]
}

with open(get_results_file_name(), "ab") as rfile:
  pickle.dump(results, rfile)

Micro Averaging. Precision: 0.4067211625794732, Recall: 0.626907461850763, F1 Score: 0.493361978736297
Macro Averaging. Precision: 0.3675190234878991, Recall: 0.5367868273260352, F1 Score: 0.4295094437724686


# Summarize Resource Utilization

In [135]:
import pickle
def get_object(path):
  obj = None
  with open(path, "rb") as p:
    obj = pickle.load(p)

  return obj

def to_gb(b):
  return round(b/(1024*1024*1024), 2)

def to_mb(b):
  return round(b/(1024*1024), 2)

In [ ]:
! ls -l "{PROJECT_PATH}/stats"

In [137]:
# system configuration
import psutil

print(f"Total GPU Memory: {to_gb(torch.cuda.get_device_properties(0).total_memory)} GB")
print(f"Total CPU Memory: {to_gb(psutil.virtual_memory().total)} GB")

Total GPU Memory: 15.9 GB
Total CPU Memory: 51.01 GB


In [138]:
import numpy as np
stats = get_object(f"{PROJECT_PATH}/stats/stats-17-04-2022-02-35-12")
print(f"Average GPU Memory Used: {to_mb(np.mean(stats['gpu_mem']))} MB")
print(f"Average Main Memory Used: {to_gb(np.mean(stats['main_mem']))} GB")
print(f"Average GPU Time (per epoc): {np.mean(stats['gpu_time']):.2f} sec")
print(f"Total GPU Time: {np.sum(stats['gpu_time']):.2f} sec")

Average GPU Memory Used: 39.68 MB
Average Main Memory Used: 10.39 GB
Average GPU Time (per epoc): 35.34 sec
Total GPU Time: 3533.87 sec


In [141]:
import numpy as np

a_mi = np.array([0.496, 0.564, 0.528])
o_mi = np.array([0.406, 0.626, 0.493])
print(((a_mi - o_mi).T / a_mi) * 100)

a_ma = np.array([0.464, 0.463, 0.448])
o_ma = np.array([0.367, 0.536, 0.429])
print(((a_ma - o_ma).T / a_ma) * 100)

[ 18.14516129 -10.9929078    6.62878788]
[ 20.90517241 -15.76673866   4.24107143]
